In [1]:
### EDA　###

import pandas as pd
import numpy as np
from IPython.display import Image, display
import matplotlib.pyplot as plt
%matplotlib inline

# データの読み込み
train_df = pd.read_csv('../data/raw/train.csv')
test_df = pd.read_csv('../data/raw/test.csv')

# 欠損値 -1 を　 NaNに置き換える
train_df[['price_am', 'price_pm']] = train_df[['price_am', 'price_pm']].replace(-1, np.nan)
test_df[['price_am', 'price_pm']] = test_df[['price_am', 'price_pm']].replace(-1, np.nan)

display(train_df.shape, test_df.shape)
display(train_df.info(), test_df.info())
display(train_df.isnull().sum(), test_df.isnull().sum())
display(train_df.describe(), test_df.describe())

(2101, 6)

(365, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  2101 non-null   object 
 1   y         2101 non-null   int64  
 2   client    2101 non-null   int64  
 3   close     2101 non-null   int64  
 4   price_am  1890 non-null   float64
 5   price_pm  1889 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 98.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datetime  365 non-null    object 
 1   client    365 non-null    int64  
 2   close     365 non-null    int64  
 3   price_am  360 non-null    float64
 4   price_pm  360 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 14.4+ KB


None

None

datetime      0
y             0
client        0
close         0
price_am    211
price_pm    212
dtype: int64

datetime    0
client      0
close       0
price_am    5
price_pm    5
dtype: int64

,y,client,close,price_am,price_pm
count,2101.000000,2101.000000,2101.000000,1890.000000,1889.000000
mean,34.096621,0.098049,0.012851,0.794709,0.546850
std,17.476234,0.297451,0.112658,1.044370,0.903766
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21.000000,0.000000,0.000000,0.000000,0.000000
50%,32.000000,0.000000,0.000000,0.500000,0.000000
75%,45.000000,0.000000,0.000000,1.000000,1.000000
max,109.000000,1.000000,1.000000,5.000000,5.000000


,client,close,price_am,price_pm
count,365.000000,365.000000,360.000000,360.000000
mean,0.452055,0.013699,0.819444,0.608333
std,0.498379,0.116396,0.986347,0.946736
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000
75%,1.000000,0.000000,1.000000,1.000000
max,1.000000,1.000000,5.000000,5.000000


In [2]:
# 訓練データとテストデータをマージ

train_df['is_train'] = 1
test_df['is_train'] = 0

train_cols = train_df.columns.tolist()
test_cols = test_df.columns.tolist()
common_cols = list(set(train_cols) & set(test_cols))
train_only = list(set(train_cols) - set(common_cols))
test_only = list(set(test_cols) - set(common_cols))

for col in train_only:
    test_df[col] = None

for col in test_only:
    train_df[col] = None

merged_df = pd.concat([train_df, test_df], ignore_index=True, sort=False)

merged_df['price'] = (merged_df['price_am'] + merged_df['price_pm']) / 2

pd.set_option('future.no_silent_downcasting', True)
merged_df['y'] = merged_df['y'].fillna(0).astype(int)

merged_df.head(190)

,datetime,y,client,close,price_am,price_pm,is_train,price
0,2010-07-01,17,0,0,NaN,NaN,1,NaN
1,2010-07-02,18,0,0,NaN,NaN,1,NaN
2,2010-07-03,20,0,0,NaN,NaN,1,NaN
3,2010-07-04,20,0,0,NaN,NaN,1,NaN
4,2010-07-05,14,0,0,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...
185,2011-01-02,0,0,1,NaN,NaN,1,NaN
186,2011-01-03,0,0,1,NaN,NaN,1,NaN
187,2011-01-04,16,0,0,0.0,0.0,1,0.0
188,2011-01-05,16,0,0,0.0,0.0,1,0.0


In [3]:
merged_df.set_index('datetime', inplace=True)
merged_df.index = pd.to_datetime(merged_df.index)
merged_df

,y,client,close,price_am,price_pm,is_train,price
datetime,,,,,,,
2010-07-01,17,0,0,NaN,NaN,1,NaN
2010-07-02,18,0,0,NaN,NaN,1,NaN
2010-07-03,20,0,0,NaN,NaN,1,NaN
2010-07-04,20,0,0,NaN,NaN,1,NaN
2010-07-05,14,0,0,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...
2017-03-27,0,0,0,5.0,4.0,0,4.5
2017-03-28,0,0,0,4.0,4.0,0,4.0
2017-03-29,0,0,0,3.0,3.0,0,3.0


In [4]:
merged_df = merged_df.loc[(merged_df.index< pd.Timestamp('2010-07-01')) |
            (merged_df.index >= pd.Timestamp('2011-01-04'))]
merged_df

,y,client,close,price_am,price_pm,is_train,price
datetime,,,,,,,
2011-01-04,16,0,0,0.0,0.0,1,0.0
2011-01-05,16,0,0,0.0,0.0,1,0.0
2011-01-06,13,0,0,0.0,0.0,1,0.0
2011-01-07,14,0,0,0.0,0.0,1,0.0
2011-01-08,16,0,0,0.0,0.0,1,0.0
...,...,...,...,...,...,...,...
2017-03-27,0,0,0,5.0,4.0,0,4.5
2017-03-28,0,0,0,4.0,4.0,0,4.0
2017-03-29,0,0,0,3.0,3.0,0,3.0


In [5]:
# ラグ特徴量の追加
merged_df['y_lag_365'] = merged_df['y'].shift(365)

for lag in [1, 7, 365]:
    merged_df[f'price_am_lag_{lag}'] = merged_df['price_am'].shift(lag)

for lag in [1, 7, 365]:
    merged_df[f'price_pm_lag_{lag}'] = merged_df['price_pm'].shift(lag)


# 移動平均の追加
windows = [7, 30]
for window in windows:
    merged_df[f'price_am_moving_avg_{window}'] = merged_df['price_am'].rolling(window=window).mean()

windows = [7, 30]
for window in windows:
    merged_df[f'price_pm_moving_avg_{window}'] = merged_df['price_pm'].rolling(window=window).mean()

/var/folders/55/p_rc8dfn457_yr0748h20c4r0000gn/T/ipykernel_4385/959168631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['y_lag_365'] = merged_df['y'].shift(365)
/var/folders/55/p_rc8dfn457_yr0748h20c4r0000gn/T/ipykernel_4385/959168631.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[f'price_am_lag_{lag}'] = merged_df['price_am'].shift(lag)
/var/folders/55/p_rc8dfn457_yr0748h20c4r0000gn/T/ipykernel_4385/959168631.py:5: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [7]:
merged_df.to_csv('processed20240826_2.csv')